In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
intents=pd.read_json('/content/intents.json',orient='index')

In [ ]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?',
    "Hello! I'm Dexter. How may I help you?",
    'Hey there!'],
   'tag': 'greeting'},
  {'context_set': '',
   'patterns': ['Book a table',
    'Can I book a table?',
    'I want to book a table',
    'Book seat',
    'I want to book a seat',
    'Can I book a seat?'],
   'responses': [''],
   'tag': 'book_table'},
  {'context_set': '',
   'patterns': ['How many seats are available?',
    'Available seats',
    'How many tables are available?',
    'Available tables'],
   'responses': [''],
   'tag': 'available_tables'},
  {'context_set': '',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day',
    'cya later',
    'I gotta go now',
    'I gotta rush now'],
   'responses': ['Sad to see you go :(

In [ ]:
import random
import json
import pickle
import nltk 
from nltk.stem  import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
with open('intents.json') as file:
    intents = json.load(file)

In [ ]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

for intent in intents['intents']:
  for pattern in intent['patterns']:
     word_list=nltk.word_tokenize(pattern)
     words.extend(word_list)
     documents.append((word_list,intent['tag']))
     if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [ ]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
classes=sorted(set(classes))


In [ ]:
training=[]
output_empty=[0]*len(classes)
for document in documents:
   bag=[]
   word_patterns=document[0]
   word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
   for word in words:
      bag.append(1) if word in word_patterns else bag.append(0)

   output_row=list(output_empty)
   output_row[classes.index(document[1])]=1
   training.append([bag,output_row])
random.shuffle(training)
training=np.array(training)

train_x=list(training[:,0])
train_y=list(training[:,1])

In [ ]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd=SGD(lr=0.01,momentum=0.9,  nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=4)
model.save('Chatbotmodel.h5',hist)
print('Done')

Epoch 1/200
14/14 [==============================] - 1s 2ms/step - loss: 2.4424 - accuracy: 0.1547
Epoch 2/200
14/14 [==============================] - 0s 2ms/step - loss: 2.3280 - accuracy: 0.1551
Epoch 3/200
14/14 [==============================] - 0s 2ms/step - loss: 2.3250 - accuracy: 0.1114
Epoch 4/200
14/14 [==============================] - 0s 2ms/step - loss: 2.1430 - accuracy: 0.3425
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 2.0829 - accuracy: 0.3155
Epoch 6/200
14/14 [==============================] - 0s 2ms/step - loss: 2.1363 - accuracy: 0.3544
Epoch 7/200
14/14 [==============================] - 0s 2ms/step - loss: 1.7601 - accuracy: 0.5388
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 1.8444 - accuracy: 0.4142
Epoch 9/200
14/14 [==============================] - 0s 2ms/step - loss: 1.4933 - accuracy: 0.6965
Epoch 10/200
14/14 [==============================] - 0s 2ms/step - loss: 1.5195 - accuracy: 0.4981
Epoch 11/

In [ ]:
from tensorflow.keras.models import load_model

model=load_model('Chatbotmodel.h5')

def clean_up_sentence(sentence):
     sentence_words=nltk.word_tokenize(sentence)
     sentence_words=[lemmatizer.lemmatize(w) for w in sentence_words]
     return sentence_words 

def bag_of_words(sentence):
    sentence_words=clean_up_sentence(sentence)
    bag=[0] * len(words)
    for w in sentence_words:
       for i,word in enumerate(words):
           if word==w:
              bag[1]=1
    return np.array(bag)          

def predict_classes(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    error_threshold=0.25
    results=[[i,r] for i,r in enumerate(res) if r >  error_threshold]
    results.sort(key=lambda x:x[1], reverse=True)
    return_list=[]
    for r in results:
        print(r)
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list



In [ ]:
def get_respone(intents_list,intents_json):
    tag=intents_list[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
           result=random.choices(i['responses'])
           break
    return result  
print("Bot is running")    

Bot is running


In [ ]:
while True:
   message=input("")
   ints=predict_classes(message)
   res=get_respone(ints[0],ints[1])
   print(res)

what is the price of a car
[6, 0.6890064]
[5, 0.2746207]


KeyError: ignored